# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-NOTEBOOK_CODE_HERE-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Politics Workflow Series ("International Treaties") 

Consider the following exploratory information need:

> You investigating existing international treaties

wd:Q46## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wdt:P39`     | position held   | predicate |
| `wd:Q131569`  | treaty | node      |
| `wd:Q46`      | Europe        | node |
| `wd:Q38`      | Italy          | node |
| `wd:Q21707860` | Paris Agreement           | node |



Also consider

```
?p wdt:P31/wdt:P279* wd:Q131569  . 
```

is the BGP to retrieve all **treaties**

## Workload Goals


1. Identify the BGP to retrieve treaties that took place in E.U. countries

2. Identify the BGP for obtaining which countries or people signed the treaties

3. How many treaties have been published per year?

4. For how many treaties we know causes or effects?

5. Analyze the number of countries and how many treaties they have signed and their causes
 
   5.1 Are there two countries that signed many common treaties?
   
   5.2 Which are the treaties with most signatories? Which country signed the most treaties?
   
   5.3 Are there countries that did not sign a treaty with the U.S. but signed a treaty with Germany?


In [2]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT(*)
WHERE { 

?p wdt:P31/wdt:P279* wd:Q131569  . 
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '3962')]


1

### 1. Identify the BGP to retrieve treaties that took place in E.U. countries

In [4]:
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
    ?sc wdt:P31/wdt:P279* wd:Q131569 ;
        ?p ?o . FILTER(!isLiteral(?o))
    ?p sc:name ?pname .
}
   
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2058'), ('pname', 'depositor')]
[('p', 'http://www.wikidata.org/prop/direct/P2568'), ('pname', 'repealed by')]
[('p', 'http://www.wikidata.org/prop/direct/P2567'), ('pname', 'amended by')]
[('p', 'http://www.wikidata.org/prop/direct/P8739'), ('pname', 'prohibits')]
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('pname', 'applies to jurisdiction')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pname', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pname', 'replaced by')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pname', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1537'), ('pname', 'contributing fa

130

In [5]:
queryString = """
SELECT ?pname
WHERE { 
    ?p wdt:P31/wdt:P279* wd:Q131569 .
    ?p sc:name ?pname .
    ?p wdt:P276 ?l. 
    ?l wdt:P30 wd:Q46.
}
LIMIT 5
"""

print("Results")
run_query(queryString)

Results
[('pname', 'Treaty of Craiova')]
[('pname', 'Anglo-Egyptian treaty of 1936')]
[('pname', 'Treaty of London')]
[('pname', 'Treaty on Open Skies')]
[('pname', 'Berne Convention for the Protection of Literary and Artistic Works')]


5

### 2. Identify the BGP for obtaining which countries or people signed the treaties

In [6]:
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
    ?sc wdt:P31/wdt:P279* wd:Q131569 ;
        ?p ?o . FILTER(!isLiteral(?o))
    ?p sc:name ?pname .
    FILTER(REGEX(?pname, ".*sign.*"))
}
   
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P793'), ('pname', 'significant event')]
[('p', 'http://www.wikidata.org/prop/direct/P1891'), ('pname', 'signatory')]
[('p', 'http://www.wikidata.org/prop/direct/P7153'), ('pname', 'significant place')]
[('p', 'http://www.wikidata.org/prop/direct/P1435'), ('pname', 'heritage designation')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pname', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P3342'), ('pname', 'significant person')]


6

In [7]:
queryString = """
SELECT ?pname (GROUP_CONCAT(DISTINCT ?loc; SEPARATOR=", "))
WHERE { 
    ?p wdt:P31/wdt:P279* wd:Q131569 .
    ?p sc:name ?pname .
    ?p wdt:P1891 ?l.
    ?l sc:name ?loc
}
GROUP BY ?pname
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('pname', 'Chanyuan Treaty'), ('callret-1', 'Liao dynasty, Northern Song Dynasty')]
[('pname', 'Treaty of Athis-sur-Orge'), ('callret-1', 'County of Flanders')]
[('pname', 'Treaty of Melno'), ('callret-1', 'Grand Duchy of Lithuania, Kingdom of Poland, Teutonic Order')]
[('pname', 'British-Brazilian Treaty of 1826'), ('callret-1', 'Empire of Brazil, United Kingdom of Great Britain and Ireland')]
[('pname', 'Nice Agreement Concerning the International Classification of Goods and Services for the Purposes of the Registration of Marks'), ('callret-1', 'Austria, Belgium, Denmark, France, Germany, Hungary, Italy, Lebanon, Liechtenstein, Luxembourg, Monaco, Morocco, Netherlands, Norway, Portugal, Romania, Spain, Sweden, Switzerland, Tunisia, Turkey, United Kingdom')]
[('pname', 'General Security Agreement between the United Kingdom of Great Britain and Northern Ireland and the Kingdom of Spain concerning the protection of classified information exchanged between the two countries'), 

10

### 3. How many treaties have been published per year?

In [8]:
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
    ?sc wdt:P31/wdt:P279* wd:Q131569 ;
        ?p ?o . FILTER(isLiteral(?o))
    ?p sc:name ?pname .
    FILTER(REGEX(?pname, ".*[D|d]ate.*"))
}
   
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1619'), ('pname', 'date of official opening')]
[('p', 'http://www.wikidata.org/prop/direct/P577'), ('pname', 'publication date')]
[('p', 'http://www.wikidata.org/prop/direct/P7588'), ('pname', 'effective date')]
[('p', 'http://www.wikidata.org/prop/direct/P576'), ('pname', 'dissolved, abolished or demolished date')]
[('p', 'http://www.wikidata.org/prop/direct/P6949'), ('pname', 'announcement date')]
[('p', 'http://www.wikidata.org/prop/direct/P569'), ('pname', 'date of birth')]


6

In [9]:
queryString = """

SELECT (year(?date)) COUNT(DISTINCT ?p)
WHERE { 
    ?p wdt:P31/wdt:P279* wd:Q131569.
    ?p wdt:P577 ?date .
} 
GROUP BY (year(?date))
ORDER BY DESC (COUNT(DISTINCT ?p))
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '1929'), ('callret-1', '20')]
[('callret-0', '1926'), ('callret-1', '18')]
[('callret-0', '1924'), ('callret-1', '18')]
[('callret-0', '1931'), ('callret-1', '17')]
[('callret-0', '1932'), ('callret-1', '16')]
[('callret-0', '2000'), ('callret-1', '16')]
[('callret-0', '1936'), ('callret-1', '13')]
[('callret-0', '1927'), ('callret-1', '11')]
[('callret-0', '1922'), ('callret-1', '9')]
[('callret-0', '1928'), ('callret-1', '9')]


10

### 4. For how many treaties we know causes or effects?

In [10]:
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
    ?sc wdt:P31/wdt:P279* wd:Q131569 ;
        ?p ?o . FILTER(!isLiteral(?o))
    ?p sc:name ?pname .
    FILTER(REGEX(?pname, ".*[C|c]ause.*") || REGEX(?pname, ".*[E|e]ffect.*"))
}
   
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1542'), ('pname', 'has effect')]
[('p', 'http://www.wikidata.org/prop/direct/P828'), ('pname', 'has cause')]
[('p', 'http://www.wikidata.org/prop/direct/P1478'), ('pname', 'has immediate cause')]


3

In [11]:
queryString = """
SELECT COUNT(distinct ?p)
WHERE {
    {
    ?p wdt:P31/wdt:P279* wd:Q131569 .
    ?p wdt:P1542 ?e.
    }
    UNION
    {
    ?p wdt:P31/wdt:P279* wd:Q131569 .
    ?p wdt:P828 ?c.
    }
}
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '53')]


1

### 5.1 Are there two countries that signed many common treaties?

###  5.2 Which are the treaties with most signatories? Which country signed the most treaties?

In [12]:
queryString = """
SELECT ?pname COUNT(DISTINCT ?loc)
WHERE { 
    ?p wdt:P31/wdt:P279* wd:Q131569 .
    ?p sc:name ?pname .
    ?p wdt:P1891 ?l.
    ?l sc:name ?loc
}
GROUP BY ?pname
ORDER BY DESC (COUNT(DISTINCT ?loc))
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('pname', 'Universal Copyright Convention'), ('callret-1', '97')]
[('pname', 'Treaty on the Prohibition of Nuclear Weapons'), ('callret-1', '86')]
[('pname', 'Marrakesh Treaty to Facilitate Access to Published Works for Persons Who Are Blind, Visually Impaired or Otherwise Print Disabled'), ('callret-1', '80')]
[('pname', 'Beijing Treaty on Audiovisual Performances'), ('callret-1', '74')]
[('pname', 'Energy Charter Treaty'), ('callret-1', '54')]
[('pname', 'Patent Law Treaty'), ('callret-1', '54')]
[('pname', 'Singapore Treaty on the Law of Trademarks'), ('callret-1', '53')]
[('pname', 'Trademark Law Treaty'), ('callret-1', '51')]
[('pname', 'World Intellectual Property Organization Copyright Treaty'), ('callret-1', '51')]
[('pname', 'Convention Establishing the World Intellectual Property Organization'), ('callret-1', '50')]


10

In [13]:
queryString = """
SELECT ?country COUNT(DISTINCT ?p)
WHERE { 
    ?p wdt:P31/wdt:P279* wd:Q131569 .
    ?p sc:name ?pname .
    ?p wdt:P1891 ?c.
    ?c sc:name ?country
}
GROUP BY ?country
ORDER BY DESC (COUNT(DISTINCT ?p))
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('country', 'France'), ('callret-1', '79')]
[('country', 'Spain'), ('callret-1', '75')]
[('country', 'United States of America'), ('callret-1', '69')]
[('country', 'Germany'), ('callret-1', '62')]
[('country', 'United Kingdom'), ('callret-1', '51')]
[('country', 'Italy'), ('callret-1', '44')]
[('country', 'Russian Empire'), ('callret-1', '43')]
[('country', 'Austria'), ('callret-1', '42')]
[('country', 'Switzerland'), ('callret-1', '42')]
[('country', 'Sweden'), ('callret-1', '40')]


10

### 5.3 Are there countries that did not sign a treaty with the U.S. but signed a treaty with Germany?